In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('data\movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1872,tt0134983,90000000,14828081,Supernova,James Spader|Angela Bassett|Robert Forster|Lou...,Walter Hill|Jack Sholder,"In the farthest reaches of space, something ha...","Set in the 22nd century, when a battered salva...",91,Science Fiction,United Artists|Metro-Goldwyn-Mayer (MGM)|Hamme...,1/14/2000,4.7,2000
1023,tt0837562,85000000,358375603,Hotel Transylvania,Adam Sandler|Steve Buscemi|David Spade|Kevin J...,Genndy Tartakovsky,Where monsters go to get away from it all,"Dracula, who operates a high-end resort away f...",91,Animation|Comedy|Family|Fantasy,Columbia Pictures|Sony Pictures Animation,9/20/2012,6.7,2012
875,tt0293662,21000000,43928932,The Transporter,Jason Statham|Shu Qi|FranÃ§ois BerlÃ©and|Matt ...,Louis Leterrier|Corey Yuen Kwai,Rules are made to be broken.,"Former Special Forces officer, Frank Martin wi...",92,Action|Crime|Thriller,TF1 Films Productions|Current Entertainment|Ca...,10/2/2002,6.5,2002
1610,tt0363547,28000000,102356381,Dawn of the Dead,Sarah Polley|Ving Rhames|Jake Weber|Mekhi Phif...,Zack Snyder,"When the undead rise, civilization will fall.",A group of surviving people take refuge in a s...,101,Fantasy|Horror|Action,New Amsterdam Entertainment|Strike Entertainme...,3/19/2004,6.7,2004
1614,tt0363988,40000000,92913171,Secret Window,Johnny Depp|John Turturro|Timothy Hutton|Charl...,David Koepp,The most important part of a story is the ending.,"Mort Rainey, a writer just emerging from a pai...",96,Thriller|Mystery,Columbia Pictures Corporation|Pariah Entertain...,3/12/2004,6.3,2004


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.describe(include = ['object'])
# для анализа наличия повторов, например наименований фильмов или уж не дай Бог id фильмов...

,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt0418763,Fantastic Four,Jennifer Lawrence|Josh Hutcherson|Liam Hemswor...,Steven Soderbergh,There are no clean getaways.,When Betty Anne Waters' older brother Kenny is...,Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


In [5]:
data.original_title.value_counts()

Fantastic Four       2
The Gift             2
Hotel Rwanda         1
The Conjuring        1
Hidalgo              1
                    ..
Our Idiot Brother    1
Killer Joe           1
Albert Nobbs         1
The Lake House       1
Diana                1
Name: original_title, Length: 1887, dtype: int64

In [6]:
data.info()
# проверим типы данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [7]:
answers = {} # создадим словарь для ответов

# добавим идентификатор содержащий название фильма
data['movie_id'] = data['original_title'] + ' (' + data['imdb_id'] + ')'

# добавим колонку с прибылью каждого фильма
data['profit'] = data['revenue'] - data['budget']

# преобразуем данные с датой врелиза в формат даты
#data['release_date'] = data['release_date'].apply(lambda d:datetime.strptime(d, '%m/%d/%Y'))
data.release_date = pd.to_datetime(data.release_date)

# добавим месяц релиза
data['release_month'] = data.release_date.dt.month

In [8]:
data.head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,movie_id,profit,release_month
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,Jurassic World (tt0369610),1363528810,6
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,Mad Max: Fury Road (tt1392190),228436354,5
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,Insurgent (tt2908446),185238201,3


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [9]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [10]:
# тут пишем ваш код для решения данного вопроса:
data.sort_values(['budget']).tail(1)['movie_id']

723    Pirates of the Caribbean: On Stranger Tides (t...
Name: movie_id, dtype: object

ВАРИАНТ 2

In [11]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [12]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [13]:
data.sort_values(['runtime']).tail(1)['movie_id']

1157    Gods and Generals (tt0279111)
Name: movie_id, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [14]:
#display(data.sort_values(['runtime']))
data.sort_values(['runtime']).head(1)['movie_id']

768    Winnie the Pooh (tt1449283)
Name: movie_id, dtype: object

In [15]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [16]:
data.runtime.mean()

109.6585494970884

In [17]:
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [18]:
data.runtime.median()

107.0

In [19]:
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [20]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data.sort_values(['profit']).tail(1)['movie_id']

239    Avatar (tt0499549)
Name: movie_id, dtype: object

In [21]:
answers['6'] = 'Avatar (tt0499549)'
# рассмотрим данный фильм поподробней
data.loc[data['movie_id'] == answers['6']]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,movie_id,profit,release_month
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,2009-12-10,7.1,2009,Avatar (tt0499549),2544505847,12


# 7. Какой фильм самый убыточный? 

In [22]:
data.sort_values(['profit']).head(1)['movie_id']

1245    The Lone Ranger (tt1210819)
Name: movie_id, dtype: object

In [23]:
answers['7'] = 'The Lone Ranger (tt1210819)'
# рассмотрим данный фильм поподробней
data.loc[data['movie_id'] == answers['7']]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,movie_id,profit,release_month
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,The Lone Ranger (tt1210819),-165710090,7


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [24]:
data.query('profit > 0').movie_id.count()

1478

In [25]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [26]:
data.query('release_year == 2008').sort_values(['revenue']).tail(1)['movie_id']

599    The Dark Knight (tt0468569)
Name: movie_id, dtype: object

In [27]:
answers['9'] = 'The Dark Knight (tt0468569)'
# рассмотрим данный фильм поподробней
data.loc[data['movie_id'] == answers['9']]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,movie_id,profit,release_month
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,The Dark Knight (tt0468569),816921825,7


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [28]:
data.query('release_year in [2012, 2013, 2014]').sort_values(['profit']).head(1)['movie_id']

1245    The Lone Ranger (tt1210819)
Name: movie_id, dtype: object

In [29]:
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [30]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
data.genres.str.split('|').explode().value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

In [31]:
answers['11'] = 'Drama'

ВАРИАНТ 2

In [32]:
data.genres.str.split('|').explode().mode()

0    Drama
dtype: object

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
data[data.profit>0].genres.str.split('|').explode().value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

In [34]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [35]:
# список режесеров
directors = set(data.director.str.split('|').sum())
pd.Series({x: data[data.director.str.contains(x)].revenue.sum() for x in directors}).sort_values(ascending = False)

Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Length: 997, dtype: int64

In [36]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [37]:
data[data.genres.str.contains('Action')].director.str.split('|').explode().value_counts()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
George Clooney        1
Sean McNamara         1
Joe Cornish           1
Sanjay Gadhvi         1
Gary McKendry         1
Name: director, Length: 364, dtype: int64

In [38]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [39]:
casts = set(data[data.release_year==2012].cast.str.split('|').sum())
pd.Series({x: data[(data.cast.str.contains(x)) & (data.release_year==2012)].revenue.sum() for x in casts}).sort_values(ascending = False)

Chris Hemsworth       2027450773
Denis Leary           1629460639
Anne Hathaway         1522851057
Mark Ruffalo          1519557910
Scarlett Johansson    1519557910
                         ...    
Michael Nyqvist          3428048
Josh Lucas               2106557
Sami Gayle               2106557
Nicolas Cage             2106557
Danny Huston             2106557
Length: 466, dtype: int64

In [40]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [41]:
data[data.budget > data.budget.mean()].cast.str.split('|').explode().value_counts()

Matt Damon               18
Adam Sandler             17
Angelina Jolie           16
Eddie Murphy             15
Tom Cruise               15
                         ..
Armin Rohde               1
Sarah Michelle Gellar     1
Anna Friel                1
Christopher Eccleston     1
Rachel Nichols            1
Name: cast, Length: 1505, dtype: int64

In [42]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [43]:
data[data.cast.str.contains('Nicolas Cage')].genres.str.split('|').explode().value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Animation           3
Mystery             3
Family              3
History             2
Romance             1
Horror              1
War                 1
Name: genres, dtype: int64

In [44]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [45]:
data[data.production_companies.str.contains('Paramount Pictures')].sort_values('profit').movie_id.head(1)

925    K-19: The Widowmaker (tt0267626)
Name: movie_id, dtype: object

In [46]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [47]:
data.groupby(['release_year']).sum().sort_values('revenue', ascending=False)

,budget,revenue,runtime,vote_average,profit,release_month
release_year,,,,,,
2015,6780630004,25449202382,13355,755.2,18668572378,842
2014,7008050000,23405862953,13444,774.5,16397812953,854
2013,7970620000,23213799791,15039,846.9,15243179791,943
2012,7002000000,23079001687,12145,682.1,16077001687,724
2011,7946550531,22676791872,15355,868.5,14730241341,949
2010,7540000000,21071204961,13592,762.1,13531204961,860
2009,6838046652,20261791024,14187,800.2,13423744372,888
2008,6588900000,18252781990,12979,731.1,11663881990,847
2007,6596495000,18162406801,14305,809.3,11565911801,911


In [48]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [49]:
data[data.production_companies.str.contains('Warner Bros')].groupby(['release_year']).sum().sort_values('profit', ascending=False)

,budget,revenue,runtime,vote_average,profit,release_month
release_year,,,,,,
2014,947600000,3243064519,1576,88.9,2295464519,89
2007,982000000,3183675217,1721,95.4,2201675217,114
2008,916000000,3050595031,1366,72.0,2134595031,81
2010,1034000000,3008712985,1421,79.6,1974712985,92
2011,974000000,2845393682,1248,68.3,1871393682,78
2003,890000000,2745493377,1085,56.7,1855493377,81
2009,915500000,2737954136,1305,67.9,1822454136,78
2013,963500000,2599953400,1100,59.9,1636453400,47
2004,1100000000,2731933725,1624,78.7,1631933725,91


In [50]:
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [51]:
pd.DatetimeIndex(data.release_date).month.value_counts()

9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: release_date, dtype: int64

In [52]:
answers['21'] = "Сентябрь"

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [53]:
months = data.release_date.dt.month
months[(months==6) | (months==7) | (months==8)].count()

450

ВАРИАНТ 2

In [54]:
data.query('release_month in [6, 7, 8]').movie_id.count()

450

In [55]:
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [56]:
data.query('release_month in [1, 2, 12]').director.str.split('|').explode().value_counts()

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Adam Shankman        4
Shawn Levy           4
                    ..
David Fincher        1
James Gunn           1
Kevin Smith          1
John Musker          1
Ron Clements         1
Name: director, Length: 358, dtype: int64

In [57]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [58]:
# будем работать с копией
data24 = data[['original_title', 'production_companies']].copy()
data24['len_title'] = data24.original_title.str.len()
# заменим симводы ряд символов в названиях студий
data24.production_companies = data24.production_companies.str.replace('(','_')
data24.production_companies = data24.production_companies.str.replace(')','_')
data24.production_companies = data24.production_companies.str.replace('&','__')
data24.production_companies = data24.production_companies.str.replace('+','___')
# список студий
production_companies_list = data24.production_companies.str.split('|').sum()
pd.Series({x: data24[data24.production_companies.str.contains(x)].len_title.mean() for x in production_companies_list}).sort_values(ascending = False)

Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Berlanti Productions           3.0
Everest Entertainment          3.0
Global Entertainment Group     2.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Length: 1771, dtype: float64

In [59]:
# если заинтересовало, чтоже это за фильмы с таким средним количеством символов в названии можно посмотреть
data[data.production_companies.str.contains('XM2 Productions')]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,movie_id,profit,release_month
1172,tt0290334,110000000,214948780,X2,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,The time has come for those who are different ...,Professor Charles Xavier and his team of genet...,133,Adventure|Action|Science Fiction|Thriller,Twentieth Century Fox Film Corporation|Donners...,2003-04-24,6.7,2003,X2 (tt0290334),104948780,4


In [60]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
# будем работать с копией
data25 = data[['overview', 'production_companies']].copy()
data25['len_overview'] = data25.overview.str.count(' ') + 1
# заменим симводы ряд символов в названиях студий
data25.production_companies = data25.production_companies.str.replace('(','_')
data25.production_companies = data25.production_companies.str.replace(')','_')
data25.production_companies = data25.production_companies.str.replace('&','__')
data25.production_companies = data25.production_companies.str.replace('+','___')
# список студий
production_companies_list = data25.production_companies.str.split('|').sum()
pd.Series({x: data25[data25.production_companies.str.contains(x)].len_overview.mean() for x in production_companies_list}).sort_values(ascending = False)

Midnight Picture Show                    175.0
Heineken Branded Entertainment           161.0
Room 9 Entertainment                     161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
Projection Pictures                       13.0
Our Stories Films                         13.0
Phantom Four                              13.0
Motion Picture Corporation of America     11.0
Empire Pictures                           11.0
Length: 1771, dtype: float64

In [62]:
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:
data.sort_values('vote_average', ascending=False)[['movie_id', 'vote_average']].head(19)

,movie_id,vote_average
599,The Dark Knight (tt0468569),8.1
118,Interstellar (tt0816692),8.0
125,The Imitation Game (tt2084970),8.0
9,Inside Out (tt2096673),8.0
34,Room (tt3170832),8.0
1183,The Wolf of Wall Street (tt0993846),7.9
128,Gone Girl (tt2267998),7.9
1191,12 Years a Slave (tt2024544),7.9
119,Guardians of the Galaxy (tt2015381),7.9
1081,The Lord of the Rings: The Return of the King ...,7.9


In [64]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
data.cast.str.split('|').apply(lambda s: list(combinations(s, 2))).explode().value_counts().head()

(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Rupert Grint, Emma Watson)            7
(Johnny Depp, Helena Bonham Carter)    6
(Ben Stiller, Owen Wilson)             6
Name: cast, dtype: int64

In [66]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

# Submission

In [67]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [68]:
# и убедиться что ни чего не пропустил)
len(answers)

27